In [2]:
import numpy as np
import pandas as pd


In [3]:
#preparing data
admission = pd.read_csv("MLkaDevta01.CSV")


In [4]:
admission.head(5)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [5]:
#preparing dummy variables
data = pd.concat([admission, pd.get_dummies(admission['rank'], prefix = "rank")] , axis=1)
data = data.drop('rank',axis=1)



In [6]:
data.head(5)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [7]:
#standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

In [8]:
data.head(5)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1


In [9]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)


In [10]:
#split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']


In [11]:
data.head(5)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,0,-0.066657,0.289305,0,1,0,0
280,0,0.625884,1.445476,0,1,0,0
33,1,1.837832,1.603135,0,0,1,0
210,0,1.318426,-0.131120,0,0,0,1
93,0,-0.066657,-1.208461,0,1,0,0


In [12]:
np.random.seed(21)
def sigmoid (x):
#calculate sigmoid
    return 1/ (1+np.exp(-x))


In [13]:
#hyperparameter
n_hidden = 2 #number of hidden units 
epoch = 900
learningrate = 0.005

In [15]:
n_records, n_features = features.shape
last_loss = None
#initialising weights
weights_input_hidden = np.random.normal(scale=1/n_features**0.5,size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5, size=n_hidden)


In [27]:
for e in range (epoch):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip (features.values , targets):
        #calculate output, forward pass
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot (hidden_output, weights_hidden_output))
        
        # backword pass
        error = y - output
                         
        #calculation error term for output unit
        output_error_term = error*output*(1-output)
                             
        #propagate error to hidden layer
                         
        #calculate hidden layer contribution to error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
                         
        #hidden error term
        hidden_error_term = hidden_error * hidden_output*(1-hidden_output)
                         
                         
        #todo update the changes in weight 
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]
    
    #update weights
    weights_input_hidden += learningrate * del_w_input_hidden / n_records
    weights_hidden_output += learningrate * del_w_hidden_output / n_records
                         
          
    # Printing out the mean square error on the training set
    if e % (epoch / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss
                         
                        

# Calculate accuracy on test data                                                                            
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))                  
                         
                         
                         
                         
                         
                         
                         
                         
                         
                         
                         
            
        
                         
                         

Train loss:  0.27628372891986996
Train loss:  0.27485693424216484
Train loss:  0.27346600280891725
Train loss:  0.2721102862255718
Train loss:  0.27078912571538577
Train loss:  0.2695018541543316
Train loss:  0.26824779798535786
Train loss:  0.2670262790132928
Train loss:  0.2658366160822918
Train loss:  0.26467812663828005
Prediction accuracy: 0.425
